In [1]:
# %env WANDB_PROJECT=counterfactual-generation
# %env WANDB_DISABLED=false
import pandas as pd
import random
import transformers
import datasets
import sklearn
import bs4
import wandb
import torch.cuda

In [2]:
def reformat_sentiment(x):
    return int(x == 'Positive')

def load_dataset(name):
    # load the dataset
    url = 'https://raw.githubusercontent.com/acmi-lab/counterfactually-augmented-data/master/sentiment/combined/paired/' + name
    dataset = pd.read_csv(url, sep='\t')
    dataset.rename(columns={"Sentiment": "sentiment", "Text": "text", "batch_id": "paired_id"}, inplace=True)
    # reformat 'sentiment' column
    dataset['sentiment'] = dataset['sentiment'].apply(lambda value: reformat_sentiment(value))

    return dataset

def random_shuffle_df(df, seed):
    random.seed(seed)
    df = sklearn.utils.shuffle(df)

    return df

def set_example_counter(idx, found_idsx):
    if idx in found_idsx:
        return 0
    else:
        found_idsx[idx] = 0
        return 1

def randomly_assign_conterfactuals(df, seed=1):
    # prepare the proper Dataframe for the dataset
    df = random_shuffle_df(df, seed)
    paired_ids = df['paired_id'].values
    found_ids = {}
    counterfactual_column = []
    for idx in paired_ids:
        counterfactual_column.append(set_example_counter(idx, found_ids))
    df['is_counterfactual'] = counterfactual_column

    return df

# prepare the dataset with input-counterfactuals instances
def prepare_dataframe_with_counterfacuals(df):

    # group by paired_id
    gb = df.groupby(by=["paired_id"])

    # create new columns "example" and "counterfactual"
    example_column = []
    counter_column = []
    paired_id_column = []
    label_ex = []
    label_counter = []
    for group_id in gb.groups: # group_id == paired_id
        group = gb.get_group(group_id)
        is_counterfactual_column = group['is_counterfactual'].values
        text_column = group['text'].values
        sentiment_column = group['sentiment'].values
        for is_counter, text, label in zip(is_counterfactual_column,
                                           text_column,
                                           sentiment_column):
            if is_counter:
                counter_column.append(text)
                label_counter.append(label)
            else:
                example_column.append(text)
                label_ex.append(label)

        paired_id_column.append(group_id)

    # clean the text from html tags
    example_column = [bs4.BeautifulSoup(el, "lxml").text for el in example_column]
    counter_column = [bs4.BeautifulSoup(el, "lxml").text for el in counter_column]

    # add the new columns to a new dataframe
    d = {'paired_id': paired_id_column,
         'example': example_column,
         'label_ex': label_ex,
         'counterfactual': counter_column,
         'label_counter': label_counter}
    df_with_counterfactuals = pd.DataFrame(data=d)
    df_with_counterfactuals.sort_values(by="paired_id", ascending=True, inplace=True)

    return  df_with_counterfactuals

In [3]:
training_set = load_dataset("train_paired.tsv")
dev_set = load_dataset("dev_paired.tsv")
test_set = load_dataset("test_paired.tsv")
print(f"Datasets are of type {type(test_set)}")
print(f"# of samples in the training set:{len(training_set)}")
print(f"# of samples in the dev set:{len(dev_set)}")
print(f"# of samples in the test set:{len(test_set)}")

Datasets are of type <class 'pandas.core.frame.DataFrame'>
# of samples in the training set:3414
# of samples in the dev set:490
# of samples in the test set:976


In [4]:
# append the 3 datasets
imdb_dataframe = pd.concat([training_set, dev_set, test_set], ignore_index=True)
print(f"# of samples:{len(imdb_dataframe)}")
imdb_dataframe.head(2)

# of samples:4880


,sentiment,text,paired_id
0,0,"Long, boring, blasphemous. Never have I been s...",4
1,1,"Long, fascinating, soulful. Never have I been ...",4


In [5]:
random_seed = 5
df_processed = randomly_assign_conterfactuals(imdb_dataframe.copy(deep=True), random_seed)
df_processed = prepare_dataframe_with_counterfacuals(df_processed)
print(f"# of samples:{len(df_processed)}")
df_processed.head(2)

# of samples:2440


,paired_id,example,label_ex,counterfactual,label_counter
0,4,"Long, boring, blasphemous. Never have I been s...",0,"Long, fascinating, soulful. Never have I been ...",1
1,13,"If you haven't seen this, it's terrible. It is...",0,"If you haven't seen this, it's incredible. It ...",1


In [6]:
# split into train-val-test (this later will be a 5-cross validation split)
random_state = 19
train_prop = 0.8 # of the whole dataset
val_prop = 0.2 # of the whole dataset
df_training = df_processed.sample(frac=train_prop, random_state=random_state)
new_val_prop = len(df_processed) * val_prop / len(df_training)

df_test = df_processed.drop(df_training.index)

df_val = df_training.sample(frac=new_val_prop, random_state=random_state)
df_training = df_training.drop(df_val.index)

print(f"# of samples for training:{len(df_training)}")
print(f"# of samples for validation:{len(df_val)}")
print(f"# of samples for test:{len(df_test)}")

# of samples for training:1464
# of samples for validation:488
# of samples for test:488


<h2>CFGs for the experiment</h2>

In [7]:
# HERE GOES A CONFIG CELL FOR THE EXPERIMENTS
run_wandb_sweep = True

MODEL_NAME = 'sshleifer/tiny-gpt2' #{gpt2 (gpt2-small, 12 layers), gpt2-medium (24 layers), gpt2-large (36 layers), gpt2-xl (48 layers)}
FREEZE_PARAMS = False
UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training
SPECIAL_TOKENS = {"bos_token": "<|BOS|>",
                  "eos_token": "<|EOS|>",
                  "unk_token": "<|UNK|>",
                  "pad_token": "<|PAD|>",
                  "sep_token": "<|SEP|>"} # or set it to None

TOKENIZE_IN_BATCH = True

SEED = 2020
WANDB_KEY='ac0eb1b13268d81f2526a3d354e135e6a1ede08c'

# set the template for prompting
TEMPLATE_PROMPT = "<bos_token><label_ex> review:<sep><example_text><sep><label_counter> review:<sep><counter_text><eos_token>"
MAP_LABELS = {0:"Negative", 1:"Positive"}

In [8]:
def wrap_with_prompt(df_row, template):
    final_text = template.replace("<label_ex>", MAP_LABELS[df_row["label_ex"]])
    final_text = final_text.replace("<example_text>", df_row["example"])
    final_text = final_text.replace("<label_counter>", MAP_LABELS[df_row["label_counter"]])
    final_text = final_text.replace("<counter_text>", df_row["counterfactual"])
    final_text = final_text.replace("<sep>", SPECIAL_TOKENS["sep_token"])
    final_text = final_text.replace("<bos_token>", SPECIAL_TOKENS["bos_token"])
    final_text = final_text.replace("<eos_token>", SPECIAL_TOKENS["eos_token"])
    return final_text

# wrap the datasets with the prompt template
df_training["wrapped_input"] = df_training.apply(lambda row: wrap_with_prompt(row, TEMPLATE_PROMPT), axis=1)
df_val["wrapped_input"] = df_val.apply(lambda row: wrap_with_prompt(row, TEMPLATE_PROMPT), axis=1)
df_test["wrapped_input"] = df_test.apply(lambda row: wrap_with_prompt(row, TEMPLATE_PROMPT), axis=1)
df_val.head(1)

,paired_id,example,label_ex,counterfactual,label_counter,wrapped_input
1357,13619,"Well , I come from Bulgaria where it 's almost...",1,"Well , I come from Bulgaria where it 's almost...",0,"<|BOS|>Positive review:<|SEP|>Well , I come fr..."


In [9]:
# Load language model objects
tokenizer = transformers.GPT2Tokenizer.from_pretrained(MODEL_NAME)
print("Downloaded tokenizer!")
if SPECIAL_TOKENS is not None:
    print(f"Len of tokenizer before adding tokens:{len(tokenizer)}")
    tokenizer.add_special_tokens(SPECIAL_TOKENS) # add special tokens
    print("Added special tokens to tokenizer!")
    print(f"Len of tokenizer after adding tokens:{len(tokenizer)}")

lm_config_class = transformers.GPT2Config.from_pretrained(MODEL_NAME)
lm = transformers.GPT2LMHeadModel.from_pretrained(MODEL_NAME, config=lm_config_class)
print("Downloaded model and cfg!")
if SPECIAL_TOKENS is not None:
    #Special tokens added, model needs to be resized accordingly
    lm.resize_token_embeddings(len(tokenizer))

# lm_config_class

Downloaded tokenizer!
Len of tokenizer before adding tokens:50257
Added special tokens to tokenizer!
Len of tokenizer after adding tokens:50261
Downloaded model and cfg!


In [10]:
# tokenizer

In [11]:
if FREEZE_PARAMS:
    for parameter in lm.parameters():
        parameter.requires_grad = False

    for i, m in enumerate(lm.transformer.h):
        #Only un-freeze the last n transformer blocks
        if i+1 > 12 - UNFREEZE_LAST_N:
            for parameter in m.parameters():
                parameter.requires_grad = True

    for parameter in lm.transformer.ln_f.parameters():
        parameter.requires_grad = True

    for parameter in lm.lm_head.parameters():
        parameter.requires_grad = True
    print(f"Freezed the first {len(lm.transformer.h)-UNFREEZE_LAST_N} model's layers")
    print(f"Only the last {UNFREEZE_LAST_N} model's layers will be trained!")
else:
    print("All the model's layers will be trained!")

All the model's layers will be trained!


In [12]:
training_set = datasets.Dataset.from_pandas(df_training)
val_set = datasets.Dataset.from_pandas(df_val)
test_set = datasets.Dataset.from_pandas(df_test)

In [13]:
# TOKENIZE datasets
def tokenize_function(examples):
    return tokenizer(examples["wrapped_input"], padding="max_length", truncation=True)
    # return tokenizer(examples["example"])

tokenized_train = training_set.map(tokenize_function, batched=TOKENIZE_IN_BATCH)
tokenized_train = tokenized_train.add_column("labels", tokenized_train['input_ids'])

tokenized_val = val_set.map(tokenize_function, batched=TOKENIZE_IN_BATCH)
tokenized_val = tokenized_val.add_column("labels", tokenized_val['input_ids'])

print("Datasets have been tokenized successfully!")

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Datasets have been tokenized successfully!


<h3>Start training</h3>

In [14]:
wandb agent cdiego89/counterfactual-generation/c3dly89q

def train():

    NO_CUDA = False
    if run_wandb_sweep:
        EPOCHS = wandb.config["EPOCHS"]
        LR = wandb.config["LR"]
        EPS = wandb.config["EPS"]
        WARMUP_STEPS = wandb.config["WARMUP_STEPS"]
        WEIGHT_DECAY = wandb.config["WEIGHT_DECAY"]
        TRAIN_BATCHSIZE = wandb.config["TRAIN_BATCHSIZE"]
        EVAL_BATCHSIZE = wandb.config["EVAL_BATCHSIZE"]
        BATCH_UPDATE = wandb.config["BATCH_UPDATE"]
    else:
        EPOCHS = 2
        LR = 5e-4
        EPS = 1e-8
        WARMUP_STEPS = 1e2
        WEIGHT_DECAY = 0.01
        TRAIN_BATCHSIZE = 1
        EVAL_BATCHSIZE = 1
        BATCH_UPDATE = 1

    training_args = transformers.TrainingArguments(
        output_dir="trained_models/",
        no_cuda=NO_CUDA,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=TRAIN_BATCHSIZE,
        per_device_eval_batch_size=EVAL_BATCHSIZE,
        gradient_accumulation_steps=BATCH_UPDATE,
        do_eval=True,
        evaluation_strategy=transformers.IntervalStrategy.EPOCH,
        warmup_steps=WARMUP_STEPS,
        learning_rate=LR,
        adam_epsilon=EPS,
        weight_decay=WEIGHT_DECAY,
        save_total_limit=1,
        save_strategy=transformers.IntervalStrategy.EPOCH,
        load_best_model_at_end=True,
        report_to=["wandb"]  # to log into wandb
    )

    trainer = transformers.Trainer(
        model=lm,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val
    )
    trainer.train()

    # loss = random.random()
    # wandb.log(dict(loss=loss))
    # print(f"Loss reported:{loss}")

# initialize WANDB logging system
wandb.login()
# wandb.init(group="example")
wandb.init()
sweep_id = "cdiego89/counterfactual-generation/u756ya48"
print(f"Sweep id:{sweep_id}")
wandb.agent(sweep_id, train, count=4)

wandb: Currently logged in as: cdiego89 (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Sweep id:cdiego89/counterfactual-generation/u756ya48


wandb: Agent Starting Run: 0kjponxa with config:
wandb: 	BATCH_UPDATE: 1
wandb: 	EPOCHS: 3
wandb: 	EPS: 0.10879943882850292
wandb: 	EVAL_BATCHSIZE: 1
wandb: 	LR: 0.4690756106746647
wandb: 	TRAIN_BATCHSIZE: 1
wandb: 	WARMUP_STEPS: 0.16044444169668887
wandb: 	WEIGHT_DECAY: 0.1042792886231348


Exception in thread Thread-18:
Traceback (most recent call last):
  File "/home/diego/miniconda3/envs/deeptransformers/lib/python3.7/site-packages/wandb/agents/pyagent.py", line 302, in _run_job
    self._function()
  File "/tmp/ipykernel_435211/477121050.py", line 5, in train
    EPOCHS = wandb.config["EPOCHS"]
  File "/home/diego/miniconda3/envs/deeptransformers/lib/python3.7/site-packages/wandb/sdk/wandb_config.py", line 131, in __getitem__
    return self._items[key]
KeyError: 'EPOCHS'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/diego/miniconda3/envs/deeptransformers/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/diego/miniconda3/envs/deeptransformers/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/diego/miniconda3/envs/deeptransformers/lib/python3.7/site-packages/wandb/agents/pyagent.py", line 307, in _run_job

In [15]:
wandb.finish()

AttributeError: 'ZMQDisplayPublisher' object has no attribute '_orig_publish'

In [ ]:
torch.cuda.is_available()

In [ ]:
len(tokenized_val['input_ids'][0])

In [ ]:
# metric = datasets.load_metric("rouge")
metric = datasets.load_metric("bleu")

In [ ]:
metric

In [ ]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained("gpt2-medium")

In [ ]:
transformers.AutoModelForCausalLM.from_pretrained("gpt2-medium")

In [17]:
for i in range(1):
    print(i)

0
